<a href="https://colab.research.google.com/github/Y07-assine/Disney-Dataset-Creation-with-Python-BeautifulSoup/blob/main/Dataset_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Disney Dataset Creation with Python BeautifulSoup 

Scrape & clean a list of disney wikipedia pages to create a dataset to further analyse

1. Get info box

Import necessary libraries

In [ ]:
from bs4 import BeautifulSoup as bs
import requests

Load the webpage

In [ ]:
page = requests.get("https://en.wikipedia.org/wiki/Toy_Story_3")

#convert to a beautiful soup object
soup = bs(page.content)

contents = soup.prettify()
contents

In [ ]:
info_box = soup.find(class_="infobox vevent")
info_rows = info_box.find_all("tr")

for row in info_rows:
  print(row.prettify())

<tr>
 <th class="infobox-above summary" colspan="2" style="font-size: 125%; font-style: italic;">
  Toy Story 3
 </th>
</tr>

<tr>
 <td class="infobox-image" colspan="2">
  <a class="image" href="/wiki/File:Toy_Story_3_poster.jpg" title="All of the toys packed close together, holding up a large numeral '3', with Buzz, who is putting a friendly arm around Woody's shoulder, and Woody holding the top of the 3.">
   <img alt="All of the toys packed close together, holding up a large numeral '3', with Buzz, who is putting a friendly arm around Woody's shoulder, and Woody holding the top of the 3." class="thumbborder" data-file-height="326" data-file-width="220" decoding="async" height="326" src="//upload.wikimedia.org/wikipedia/en/6/69/Toy_Story_3_poster.jpg" width="220"/>
  </a>
  <div class="infobox-caption">
   Theatrical release poster
  </div>
 </td>
</tr>

<tr>
 <th class="infobox-label" scope="row" style="white-space: nowrap; padding-right: 0.65em;">
  Directed by
 </th>
 <td class="

In [ ]:
def get_content_value(row):
  if row.find('li'):
    return [li.get_text(" ",strip=True).replace("\xa0"," ") for li in row.find_all('li')]
  else:
    return row.get_text(" ",strip=True).replace("\xa0"," ") 


movie_info = {}

for index,row in enumerate(info_rows):
  if index == 0:
    movie_info['title'] = row.find("th").get_text(" ",strip=True)
  elif index ==1:
    continue
  else:
    key = row.find("th").get_text(" ",strip=True)
    value = get_content_value(row.find("td"))
    movie_info[key] = value

movie_info

{'Box office': '$1.067 billion [1]',
 'Budget': '$200 million [1]',
 'Cinematography': ['Jeremy Lasky', 'Kim White'],
 'Country': 'United States',
 'Directed by': 'Lee Unkrich',
 'Distributed by': 'Walt Disney Studios Motion Pictures',
 'Edited by': 'Ken Schretzmann',
 'Language': 'English',
 'Music by': 'Randy Newman',
 'Produced by': 'Darla K. Anderson',
 'Production companies': ['Walt Disney Pictures', 'Pixar Animation Studios'],
 'Release date': ['June 12, 2010 ( 2010-06-12 ) ( Taormina Film Fest )',
  'June 18, 2010 ( 2010-06-18 ) (United States)'],
 'Running time': '103 minutes [1]',
 'Screenplay by': 'Michael Arndt',
 'Starring': ['Tom Hanks',
  'Tim Allen',
  'Joan Cusack',
  'Don Rickles',
  'Wallace Shawn',
  'John Ratzenberger',
  'Estelle Harris',
  'Ned Beatty',
  'Michael Keaton',
  'Jodi Benson',
  'John Morris'],
 'Story by': ['John Lasseter', 'Andrew Stanton', 'Lee Unkrich'],
 'title': 'Toy Story 3'}

2. Get info box for all movies

In [ ]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")

#convert to a beautiful soup object
soup = bs(page.content)

contents = soup.prettify()
contents

In [ ]:
movies = soup.select(".wikitable.sortable i")
movies[0:5]

[<i><a href="/wiki/Academy_Award_Review_of_Walt_Disney_Cartoons" title="Academy Award Review of Walt Disney Cartoons">Academy Award Review of Walt Disney Cartoons</a></i>,
 <i><a href="/wiki/Snow_White_and_the_Seven_Dwarfs_(1937_film)" title="Snow White and the Seven Dwarfs (1937 film)">Snow White and the Seven Dwarfs</a></i>,
 <i><a href="/wiki/Pinocchio_(1940_film)" title="Pinocchio (1940 film)">Pinocchio</a></i>,
 <i><a href="/wiki/Fantasia_(1940_film)" title="Fantasia (1940 film)">Fantasia</a></i>,
 <i><a href="/wiki/The_Reluctant_Dragon_(1941_film)" title="The Reluctant Dragon (1941 film)">The Reluctant Dragon</a></i>]

In [ ]:
def get_content_value(row):
  if row.find('li'):
    return [li.get_text(" ",strip=True).replace("\xa0"," ") for li in row.find_all('li')]
  elif row.find("br"):
    return [text for text in row.find_all('a')]
  else:
    return row.get_text(" ",strip=True).replace("\xa0"," ") 

def clean_tags(soup):
  for tag in soup.find_all(["sup","span"]):
    tag.decompose()

def get_info_box(url):

  page = requests.get(url)
  #convert to a beautiful soup object
  soup = bs(page.content)

  info_box = soup.find(class_="infobox vevent")
  info_rows = info_box.find_all("tr")

  clean_tags(soup)

  movie_info = {}

  for index,row in enumerate(info_rows):
    if index == 0:
      movie_info['title'] = row.find("th").get_text(" ",strip=True)
    else:
      header = row.find("th")
      if header:
        key = row.find("th").get_text(" ",strip=True)
        value = get_content_value(row.find("td"))
        movie_info[key] = value

  return movie_info

In [ ]:
get_info_box("https://en.wikipedia.org/wiki/The_Happiest_Millionaire")

{'Based on': [],
 'Box office': '$5 million (U.S./Canada rentals)',
 'Budget': '$5 million',
 'Cinematography': 'Edward Colman',
 'Country': 'United States',
 'Directed by': 'Norman Tokar',
 'Distributed by': 'Buena Vista Distribution',
 'Edited by': 'Cotton Warburton',
 'Language': 'English',
 'Music by': 'Jack Elliott',
 'Produced by': [<a href="/wiki/Walt_Disney" title="Walt Disney">Walt Disney</a>,
  <a href="/wiki/Bill_Anderson_(producer)" title="Bill Anderson (producer)">Bill Anderson</a>],
 'Production company': 'Walt Disney Productions',
 'Release date': ['June 23, 1967', 'November 30, 1967'],
 'Running time': [<a href="/wiki/Los_Angeles" title="Los Angeles">Los Angeles</a>,
  <a href="/wiki/New_York_City" title="New York City">New York City</a>,
  <a href="/wiki/Director%27s_cut" title="Director's cut">Director's cut</a>],
 'Screenplay by': 'A. J. Carothers',
 'Starring': ['Fred MacMurray',
  'Tommy Steele',
  'Greer Garson',
  'Gladys Cooper',
  'Geraldine Page',
  'Hermione 

In [ ]:
page = requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")

#convert to a beautiful soup object
soup = bs(page.content)
movies = soup.select(".wikitable.sortable i")
base_path = "https://en.wikipedia.org"
movie_info_list = []
for index,movie in enumerate(movies):
  try:
    relative_path = movie.a['href']
    title = movie.a['title']
    full_path = base_path + relative_path
    movie_info_list.append(get_info_box(full_path))

  except Exception as e:
    print(movie.get_text())
    print(e)

Zorro the Avenger
'NoneType' object has no attribute 'find'
The Sign of Zorro
'NoneType' object has no attribute 'find'
True-Life Adventures
'NoneType' object has no attribute 'find_all'
Escape from the Dark
'NoneType' object is not subscriptable
The London Connection
'NoneType' object has no attribute 'find'
The Omega Connection
'NoneType' object is not subscriptable
Trail of the Panda
'NoneType' object is not subscriptable
Growing Up Wild
'NoneType' object is not subscriptable
Expedition China
'NoneType' object is not subscriptable
Searcher Clade
'NoneType' object is not subscriptable
Rise
'NoneType' object has no attribute 'find_all'
Sister Act 3
'NoneType' object has no attribute 'find'
Song for a Whale
'NoneType' object is not subscriptable
Spooked
'NoneType' object is not subscriptable
Tower of Terror
'NoneType' object has no attribute 'find_all'
Tron: Ares
'NoneType' object has no attribute 'find'
The Lion King
'NoneType' object is not subscriptable
29 Dates
'NoneType' object is

In [ ]:
movie_info_list[0]

{'Box office': '$45.472',
 'Country': 'United States',
 'Distributed by': 'RKO Radio Pictures',
 'Language': 'English',
 'Production company': 'Walt Disney Productions',
 'Release date': ['May 19, 1937'],
 'Running time': '41 minutes (74 minutes 1966 release)',
 'title': 'Academy Award Review of'}

##Save/Reload Movies data

In [24]:
import json

def save_data(file,data):
  with open(file, 'w', encoding='utf-8') as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

def load_data(file):
  with open(file, encoding='utf-8') as f:
    return json.load(f)

In [ ]:
save_data('disney_data_cleaned.json',movie_info_list)

##Clean our data

In [ ]:
movie_info_list = load_data("/content/disney_data_cleaned.json")

1. Clean up references [1],[2]...

In [ ]:
def clean_tags(soup):
  for tag in soup.find_all(["sup","span"]):
    tag.decompose()

##add clean_tags function to get_info_box

2. Split up the long string

In [ ]:
#add new condition in get_content_value function 

#elif row.find("br"):
    #return [text.get_text() for text in row.find_all('a')]


3. Convert "Running time" field to integer

In [ ]:
print([movie.get('Running time','N/A') for movie in movie_info_list])

['41 minutes (74 minutes 1966 release)', '83 minutes', '88 minutes', '126 minutes', '74 minutes', '64 minutes', '70 minutes', '42 minutes', '70 min', '71 minutes', '75 minutes', '94 minutes', '73 minutes', '75 minutes', '82 minutes', '68 minutes', '74 minutes', '96 minutes', '75 minutes', '84 minutes', '77 minutes', '92 minutes', '69 minutes', '81 minutes', [], '127 minutes', '92 minutes', '76 minutes', '75 minutes', '73 minutes', '85 minutes', '81 minutes', '70 minutes', '90 min.', '80 minutes', '75 minutes', '83 minutes', '83 minutes', '72 minutes', '97 minutes', '75 minutes', '104 minutes', '93 minutes', '105 minutes', '95 minutes', '97 minutes', '134 minutes', '69 minutes', '92 minutes', '131 minutes', '79 minutes', '97 minutes', '128 minutes', '74 minutes', '91 minutes', '105 minutes', '98 minutes', '130 minutes', '89 min.', '93 minutes', '67 minutes', '98 minutes', '100 minutes', '118 minutes', '103 minutes', '110 minutes', '80 min.', '79 minutes', '91 minutes', '91 minutes', '97

In [ ]:
def minutes_to_integer(time):

  if time == "N/A":
    return None
  if isinstance(time,list):
    if len(time) >0:
      return int(time[0].split(" ")[0])
    else:
      return None
  else:
    return int(time.split(" ")[0])


In [ ]:
for movie in movie_info_list:
  if movie["title"] == "The Happiest Millionaire":
    movie["Running time"] = "164 minutes"

In [ ]:
for movie in movie_info_list:
  movie['Running time (int)'] = minutes_to_integer(movie.get('Running time','N/A'))

In [ ]:
movie_info_list[0]

{'Box office': '$45.472',
 'Country': 'United States',
 'Distributed by': 'RKO Radio Pictures',
 'Language': 'English',
 'Production company': 'Walt Disney Productions',
 'Release date': ['May 19, 1937'],
 'Running time': '41 minutes (74 minutes 1966 release)',
 'Running time (int)': 41,
 'title': 'Academy Award Review of'}

In [ ]:
print([movie.get('Running time (int)') for movie in movie_info_list])

[41, 83, 88, 126, 74, 64, 70, 42, 70, 71, 75, 94, 73, 75, 82, 68, 74, 96, 75, 84, 77, 92, 69, 81, None, 127, 92, 76, 75, 73, 85, 81, 70, 90, 80, 75, 83, 83, 72, 97, 75, 104, 93, 105, 95, 97, 134, 69, 92, 131, 79, 97, 128, 74, 91, 105, 98, 130, 89, 93, 67, 98, 100, 118, 103, 110, 80, 79, 91, 91, 97, 118, 139, 131, 92, 87, 116, 93, 110, 110, 131, 101, 108, 84, 78, 75, 164, 106, 110, 99, 113, 108, 112, 93, 91, 93, 100, 100, 79, 96, 113, 89, None, 92, 88, 92, 87, 93, 93, 93, 90, 83, 96, 88, 89, 91, 93, 92, 97, 100, 100, 89, 91, 112, 115, 95, 91, 95, 104, 74, 48, 77, 104, 128, 101, 94, 104, 90, 100, 88, 93, 98, 112, 84, 97, 97, 114, 96, 97, 109, 83, 90, 107, 96, 103, 91, 95, 105, 113, 80, 101, 90, 74, 90, 89, 110, 74, 93, 84, 83, 74, 77, 107, 93, 88, 108, 84, 121, 89, 104, 90, 86, 84, 108, 107, 96, 98, 105, 108, 94, 106, 102, 88, 102, 102, 97, 111, 100, 96, 98, 78, 81, 108, 89, 99, 89, 81, 92, 100, 89, 79, 91, 101, 104, 103, 86, 105, 75, 93, 92, 98, 95, 93, 87, 93, 87, 128, 86, 95, 114, 93,

4. Convert Budget & box office to numbers

In [ ]:
print([movie.get('Budget') for movie in movie_info_list])

[None, '$1.49 million', '$2.6 million', '$2.28 million', '$600,000', '$950,000', '$858,000', None, '$788,000', None, '$1.35 million', '$2.125 million', None, '$1.5 million', '$1.5 million', None, '$2.2 million', '$1,800,000', '$3 million', None, '$4 million', '$2 million', '$300,000', '$1.8 million', None, '$5 million', None, '$4 million', None, None, None, None, None, None, '$700,000', None, None, None, None, None, '$6 million', 'under $1 million or $1,250,000', None, '$2 million', None, None, '$2.5 million', None, None, '$4 million', '$3.6 million', None, None, None, None, '$3 million', None, '$3 million', None, None, None, None, None, None, None, None, None, '$3 million', None, None, None, None, '$4.4–6 million', None, None, None, None, None, None, None, None, None, None, None, '$4 million', None, '$5 million', None, None, None, None, '$5 million', None, None, None, None, None, None, '$4 million', None, None, None, '$6.3 million', None, None, None, None, None, None, None, None, '$5 

In [ ]:
import re

In [ ]:
import re

amounts = r"thousand|million|billion"
number = r"\d+(,\d{3})*\.*\d*"

word_re = rf"\${number}(-|\sto\s)?({number})?\s({amounts})"
value_re = rf"\${number}"

def word_to_value(word):
  value_dict = {"thousand":1000,"million":1000000,"billion":1000000000}
  return value_dict[word]

def parse_word_syntax(string):
  value_string = re.search(number,string).group()
  value = float(value_string.replace(",",""))
  word = re.search(amounts,string,flags=re.I).group()
  word = word.lower()
  word_value = word_to_value(word)
  return value*word_value

def parse_value_syntax(string):
  value_string = re.search(number,string).group()
  value = float(value_string.replace(",",""))
  return value

def money_conversion(money):
    if money == "N/A":
      return None

    if isinstance(money,list):
      if len(money)>0:
        money = money[0]
      else:
        return None

    word_syntax = re.search(word_re,money,flags=re.I)
    value_syntax = re.search(value_re,money)
  
    if word_syntax:
      return parse_word_syntax(word_syntax.group())
    elif value_syntax:
      return parse_value_syntax(value_syntax.group())
    else:
      return None


print(money_conversion("$3 Million"))

3000000.0


In [ ]:
for movie in movie_info_list:
  movie['Budget (float)'] = money_conversion(movie.get('Budget','N/A'))
  movie['Box office (float)'] = money_conversion(movie.get('Box office','N/A'))

In [ ]:
movie_info_list[-10]

{'Based on': ['Bambi, a Life in the Woods', 'Felix Salten'],
 'Box office': '$267.4 million',
 'Box office (float)': 267399999.99999997,
 'Budget': '$858,000',
 'Budget (float)': 858000.0,
 'Country': 'United States',
 'Directed by': ['David Hand', 'James Algar'],
 'Distributed by': 'RKO Radio Pictures',
 'Language': 'English',
 'Music by': ['Frank Churchill', 'Edward H. Plumb'],
 'Produced by': 'Walt Disney',
 'Production company': 'Walt Disney Productions',
 'Release date': ['August 9, 1942 (World Premiere – London)',
  'August 13, 1942 (Premiere – New York City)',
  'August 21, 1942 (U.S.)'],
 'Running time': '70 minutes',
 'Running time (int)': 70,
 'Starring': 'see below',
 'Story by': ['Perce Pearce',
  'Larry Morey',
  'Vernon Stallings',
  'Melvin Shaw',
  'Carl Fallberg',
  'Ralph Wright'],
 'title': 'Bambi'}

5. Convert dates into datetime object

In [ ]:
print([movie.get('Release date') for movie in movie_info_list])

[['May 19, 1937'], ['December 21, 1937 ( Carthay Circle Theatre )'], ['February 7, 1940 ( Center Theatre )', 'February 23, 1940 (United States)'], ['November 13, 1940'], ['June 27, 1941'], ['October 23, 1941 (New York City)', 'October 31, 1941 (U.S.)'], ['August 9, 1942 (World Premiere – London)', 'August 13, 1942 (Premiere – New York City)', 'August 21, 1942 (U.S.)'], ['August 24, 1942 (World Premiere – Rio de Janeiro)', 'February 6, 1943 (U.S. Premiere – Boston)', 'February 19, 1943 (U.S.)'], ['July 17, 1943'], ['December 21, 1944 (Mexico City)', 'February 3, 1945 (US)'], ['April 20, 1946 (New York City premiere)', 'August 15, 1946 (U.S.)'], ['November 12, 1946 (Premiere: Atlanta, Georgia)', 'November 20, 1946', 'March 30, 1947 (Stanford Theatre, Palo Alto, California)'], ['September 27, 1947'], 'May 27, 1948', [], ['October 5, 1949'], ['February 15, 1950 (Boston)', 'March 4, 1950 (United States)'], ['June 22, 1950 (World Premiere-London)', 'July 29, 1950 (US)'], ['July 26, 1951 (Lon

In [ ]:
from datetime import datetime

dates = [movie.get('Release date','N/A') for movie in movie_info_list]

def clean_date(date):
  return date.split("(")[0].strip()

def date_conversion(date):
  if isinstance(date,list):
    if len(date)>0:
        date = date[0]
    else:
        return None

  date_str = clean_date(date)
  fmts = ["%B %d, %Y","%d %B %Y"]
  for fmt in fmts:
    try:
      return datetime.strptime(date_str,fmt)
    except:
      pass
  return None


In [ ]:
for movie in movie_info_list:
  movie['Release date (datetime)'] = date_conversion(movie.get('Release date','N/A'))

In [ ]:
movie_info_list[10]

{'Based on': ['Casey at the Bat',
  'Ernest Thayer',
  'Peter and the Wolf',
  'Sergei Prokofiev'],
 'Box office': '$3.275 million (worldwide rentals)',
 'Box office (float)': 3275000.0,
 'Budget': '$1.35 million',
 'Budget (float)': 1350000.0,
 'Country': 'United States',
 'Directed by': ['Jack Kinney',
  'Clyde Geronimi',
  'Hamilton Luske',
  'Joshua Meador',
  'Robert Cormack'],
 'Distributed by': 'RKO Radio Pictures, Inc.',
 'Language': 'English',
 'Music by': ['Eliot Daniel',
  'Ken Darby',
  'Charles Wolcott',
  'Oliver Wallace',
  'Edward Plumb'],
 'Produced by': 'Walt Disney',
 'Production company': 'Walt Disney Productions',
 'Release date': ['April 20, 1946 (New York City premiere)',
  'August 15, 1946 (U.S.)'],
 'Release date (datetime)': datetime.datetime(1946, 4, 20, 0, 0),
 'Running time': '75 minutes',
 'Running time (int)': 75,
 'Starring': 'Nelson Eddy',
 'Story by': ['James Bodrero',
  'Homer Brightman',
  'Erwin Graham',
  'Eric Gurney',
  'T. Hee',
  'Sylvia Hollan

##Saving data using pickle

In [4]:
import pickle

def save_data_pickle(name,data):
  with open(name,'wb') as f:
    pickle.dump(data,f)

def load_data_pickle(name):
  with open(name,'rb') as f:
    return pickle.load(f)

In [ ]:
save_data_pickle("disney_movie_data_cleaned.pickle",movie_info_list)

In [5]:
data = load_data_pickle("disney_movie_data_cleaned.pickle")

###Attach IMDB/ROTTEN Tomatoes scores

In [14]:
import requests
import os
import urllib

def get_ombd_info(title):
  base_url = "http://www.omdbapi.com/?"
  parameters = {"apikey":os.environ['OMDB_API_KEY'],'t':title}
  params_encoded = urllib.parse.urlencode(parameters)
  full_url = base_url + params_encoded
  return requests.get(full_url).json()

def get_rotten_tomato_score(omdb):
  ratings = omdb.get('Ratings',[])
  for rating in ratings:
    if rating['Source'] == 'Rotten Tomatoes':
      return rating['Value']
  
  return None
  
info = get_ombd_info('into the woods')
get_rotten_tomato_score(info)

'71%'

In [17]:
for movie in data:
  title = movie['title']
  omdb_info = get_ombd_info(title)
  movie['imdb'] = omdb_info.get('imdbRating',None)
  movie['metascore'] = omdb_info.get('Metascore',None)
  movie['rotten_tomatoes'] = get_rotten_tomato_score(omdb_info)

In [18]:
data[10]

{'Based on': ['Casey at the Bat',
  'Ernest Thayer',
  'Peter and the Wolf',
  'Sergei Prokofiev'],
 'Box office': '$3.275 million (worldwide rentals)',
 'Box office (float)': 3275000.0,
 'Budget': '$1.35 million',
 'Budget (float)': 1350000.0,
 'Country': 'United States',
 'Directed by': ['Jack Kinney',
  'Clyde Geronimi',
  'Hamilton Luske',
  'Joshua Meador',
  'Robert Cormack'],
 'Distributed by': 'RKO Radio Pictures, Inc.',
 'Language': 'English',
 'Music by': ['Eliot Daniel',
  'Ken Darby',
  'Charles Wolcott',
  'Oliver Wallace',
  'Edward Plumb'],
 'Produced by': 'Walt Disney',
 'Production company': 'Walt Disney Productions',
 'Release date': ['April 20, 1946 (New York City premiere)',
  'August 15, 1946 (U.S.)'],
 'Release date (datetime)': datetime.datetime(1946, 4, 20, 0, 0),
 'Running time': '75 minutes',
 'Running time (int)': 75,
 'Starring': 'Nelson Eddy',
 'Story by': ['James Bodrero',
  'Homer Brightman',
  'Erwin Graham',
  'Eric Gurney',
  'T. Hee',
  'Sylvia Hollan

In [20]:
save_data_pickle("disney_movie_data_final_result.pickle",data)

##Save data as JSON & CSV

In [21]:
data_copy = [movie.copy() for movie in data]

In [22]:
for movie in data_copy:
  date = movie['Release date (datetime)']
  if date:
    movie['Release date (datetime)'] = date.strftime("%B %d, %Y")
  else:
    movie['Release date (datetime)'] = None

In [25]:
save_data('disney_data_final_result.json',data_copy)

Convert data to Csv

In [26]:
import pandas as pd

df = pd.DataFrame(data)

In [27]:
df.head()

,title,Production company,Distributed by,Release date,Running time,Country,Language,Box office,Running time (int),Budget (float),Box office (float),Release date (datetime),imdb,metascore,rotten_tomatoes,Directed by,Written by,Based on,Produced by,Starring,Music by,Budget,Story by,Narrated by,Cinematography,Edited by,Languages,Screenplay by,Countries,Production companies,Color process,Japanese,Hepburn,Adaptation by,Animation by,Traditional,Simplified,Original title,Music,Lyrics,Book,Basis,Productions,Awards
0,Academy Award Review of,Walt Disney Productions,RKO Radio Pictures,"[May 19, 1937]",41 minutes (74 minutes 1966 release),United States,English,$45.472,41.0,NaN,4.547200e+01,1937-05-19,7.1,N/A,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Snow White and the Seven Dwarfs,Walt Disney Productions,RKO Radio Pictures,"[December 21, 1937 ( Carthay Circle Theatre )]",83 minutes,United States,English,$418 million,83.0,1490000.0,4.180000e+08,1937-12-21,7.6,95,None,"[David Hand, William Cottrell, Wilfred Jackson...","[Ted Sears, Richard Creedon, Otto Englander, D...","[Snow White, Brothers Grimm]",Walt Disney,"[Adriana Caselotti, Lucille La Verne, Harry St...","[Frank Churchill, Paul Smith, Leigh Harline]",$1.49 million,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Pinocchio,Walt Disney Productions,RKO Radio Pictures,"[February 7, 1940 ( Center Theatre ), February...",88 minutes,United States,English,$164 million,88.0,2600000.0,1.640000e+08,1940-02-07,7.4,99,100%,"[Ben Sharpsteen, Hamilton Luske, Bill Roberts,...",NaN,"[The Adventures of Pinocchio, Carlo Collodi]",Walt Disney,"[Cliff Edwards, Dickie Jones, Christian Rub, W...","[Leigh Harline, Paul J. Smith]",$2.6 million,"[Ted Sears, Otto Englander, Webb Smith, Willia...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Fantasia,Walt Disney Productions,RKO Radio Pictures,"[November 13, 1940]",126 minutes,United States,English,$76.4–$83.3 million (United States and Canada),126.0,2280000.0,8.330000e+07,1940-11-13,7.7,96,95%,"[Samuel Armstrong, James Algar, Bill Roberts, ...",NaN,NaN,"[Walt Disney, Ben Sharpsteen]","[Leopold Stokowski, Deems Taylor]",See program,$2.28 million,"[Joe Grant, Dick Huemer]",Deems Taylor,James Wong Howe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,The Reluctant Dragon,Walt Disney Productions,RKO Radio Pictures,"[June 27, 1941]",74 minutes,United States,English,"$960,000 (worldwide rentals)",74.0,600000.0,9.600000e+05,1941-06-27,6.9,N/A,None,"[Alfred Werker, Hamilton Luske, Jack Cutting, ...","[Ted Sears, Al Perkins, Bill Cottrell, Harry C...",NaN,Walt Disney,"[Robert Benchley, Frances Gifford, Nana Bryant]","[Frank Churchill, Larry Morey]","$600,000",NaN,NaN,Bert Glennon,Paul Weatherwax,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
df.to_csv("disney_data_final_result.csv")

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 506 entries, 0 to 505
Data columns (total 44 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   title                    506 non-null    object        
 1   Production company       207 non-null    object        
 2   Distributed by           503 non-null    object        
 3   Release date             496 non-null    object        
 4   Running time             487 non-null    object        
 5   Country                  451 non-null    object        
 6   Language                 484 non-null    object        
 7   Box office               398 non-null    object        
 8   Running time (int)       481 non-null    float64       
 9   Budget (float)           302 non-null    float64       
 10  Box office (float)       383 non-null    float64       
 11  Release date (datetime)  485 non-null    datetime64[ns]
 12  imdb                     481 non-nul